Concerting the entity tags of the train, test, and dev data to using BIO-tags.

In [1]:
import pandas as pd
from transformers import AutoTokenizer

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
bert_model_name = "bert-base-multilingual-cased"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

In [3]:
train = pd.read_parquet("data/train.parquet")
dev = pd.read_parquet("data/dev.parquet")
test = pd.read_parquet("data/test.parquet")

In [4]:
train.head()

,text,ents,sents,tokens,spans,dagw_source,dagw_domain,dagw_source_full
0,Danmark skal bygges af maskinernes forsigtige ...,"[{'start': 0, 'end': 7, 'label': 'GPE'}]","[{'start': 0, 'end': 60}]","[{'id': 0, 'start': 0, 'end': 7}, {'id': 1, 's...",{'incorrect_spans': []},danavis,News,Danish daily newspapers
1,Hvil i Cap Dag - 2011 år Cap D'Agde Privat bil...,"[{'start': 17, 'end': 21, 'label': 'DATE'}, {'...","[{'start': 0, 'end': 79}]","[{'id': 0, 'start': 0, 'end': 4}, {'id': 1, 's...",{'incorrect_spans': []},cc,Web,Common Crawl
2,Måtte det nye år gøre dem mere fortrolige med ...,"[{'start': 6, 'end': 16, 'label': 'DATE'}, {'s...","[{'start': 0, 'end': 171}]","[{'id': 0, 'start': 0, 'end': 5}, {'id': 1, 's...",{'incorrect_spans': []},naat,Conversation,NAAT
3,Vi har også 360 graders Private Banking rådgiv...,"[{'start': 12, 'end': 23, 'label': 'QUANTITY'}]","[{'start': 0, 'end': 117}]","[{'id': 0, 'start': 0, 'end': 2}, {'id': 1, 's...",{'incorrect_spans': []},cc,Web,Common Crawl
4,| Nyhedsmails,[],"[{'start': 0, 'end': 13}]","[{'id': 0, 'start': 0, 'end': 1}, {'id': 1, 's...",{'incorrect_spans': []},cc,Web,Common Crawl


In [5]:
my_list = ['ko', 'gris']
for i, animal in enumerate(my_list):
    print(i, animal)

0 ko
1 gris


In [6]:
# checking if two similar entities follow right after each other
count = 0
for i, row in enumerate(train.iloc):
    ents = row['ents']
    prev_ent = {'start': 0, 'end': 0, 'label': 'O'}

    for ent in ents:
        prev_end = prev_ent['end']
        prev_label = prev_ent['label']
        ent_start = ent['start']
        ent_label = ent['label']

        if ent_start == prev_end + 2 and ent_label == prev_label:
            # print(i)
            count += 1

        prev_ent = ent

print(count)

173


In [21]:
for i in range(5):
    for j in range(5):
        prod = i*j
        if j > 3:
            break
        else:
            print(prod)

0
0
0
0
0
1
2
3
0
2
4
6
0
3
6
9
0
4
8
12


In [39]:
my_list = ['ko', 'hest', 'hund']
for animal in my_list:
    if len(animal) > 1:
        print(animal)
    
    elif len(animal) > 2:
        print(animal)

ko
hest
hund


In [45]:
test_idx = 128
text = train.iloc[test_idx]['text']
ents = train.iloc[test_idx]['ents']
tokens = train.iloc[test_idx]['tokens']
print(text)
print(ents)
print(tokens)
print(text[13:26])
print(text[27:29])
print(text[31:35])
print(text[36:44])


Fysisk butik Bredskiftevej 16, 8210 Aarhus V
[{'start': 13, 'end': 26, 'label': 'FACILITY'}
 {'start': 27, 'end': 29, 'label': 'CARDINAL'}
 {'start': 31, 'end': 35, 'label': 'CARDINAL'}
 {'start': 36, 'end': 44, 'label': 'LOCATION'}]
[{'id': 0, 'start': 0, 'end': 6} {'id': 1, 'start': 7, 'end': 12}
 {'id': 2, 'start': 13, 'end': 26} {'id': 3, 'start': 27, 'end': 29}
 {'id': 4, 'start': 29, 'end': 30} {'id': 5, 'start': 31, 'end': 35}
 {'id': 6, 'start': 36, 'end': 42} {'id': 7, 'start': 43, 'end': 44}]
Bredskiftevej
16
8210
Aarhus V


In [44]:
test_idx = 128
text = train.iloc[test_idx]['text']
ents = train.iloc[test_idx]['ents']
print(text)
print(ents)
print(text[13:26])
print(text[27:29])
print(text[31:35])
print(text[36:44])

tokenized_text = bert_tokenizer(text).tokens()
no_special_tokens = tokenized_text[1:-1]
print(tokenized_text)
print(no_special_tokens)
pure_tokens = []

for i, token in enumerate(no_special_tokens):
    first_letter = token[0]

    if first_letter == '#':
        pure_token = token.strip('#')
        combined_token = pure_tokens[-1] + pure_token
        pure_tokens.pop()
        pure_tokens.append(combined_token)

    else:
        pure_tokens.append(token)

print(pure_tokens)

count = 0
token_count = 0
new_ents = []

for i,ent in enumerate(ents):
    print(f"{i+1} of {len(ents)}")
    ent_start = ent['start']
    ent_end = ent['end']
    ent_label = ent['label']

    for token in pure_tokens[token_count:]:
        print(token)
        print(f'count: {count}')
        tok_len = len(token)
        new_count = count + tok_len

        if count > ent_end:
            break

        if new_count < ent_start:
            new_ents.append('O')

        elif count == ent_start:
            new_ents.append('B-' + ent_label)

        elif new_count <= ent_end:
            new_ents.append('I-' + ent_label)

        token_count += 1
        count = new_count + 1
        print(f'count: {count}\n')

print(new_ents)

Fysisk butik Bredskiftevej 16, 8210 Aarhus V
[{'start': 13, 'end': 26, 'label': 'FACILITY'}
 {'start': 27, 'end': 29, 'label': 'CARDINAL'}
 {'start': 31, 'end': 35, 'label': 'CARDINAL'}
 {'start': 36, 'end': 44, 'label': 'LOCATION'}]
Bredskiftevej
16
8210
Aarhus V
['[CLS]', 'F', '##ysis', '##k', 'but', '##ik', 'Br', '##eds', '##ki', '##fte', '##vej', '16', ',', '821', '##0', 'Aarhus', 'V', '[SEP]']
['F', '##ysis', '##k', 'but', '##ik', 'Br', '##eds', '##ki', '##fte', '##vej', '16', ',', '821', '##0', 'Aarhus', 'V']
['Fysisk', 'butik', 'Bredskiftevej', '16', ',', '8210', 'Aarhus', 'V']
1 of 4
Fysisk
count: 0
count: 7

butik
count: 7
count: 13

Bredskiftevej
count: 13
count: 27

16
count: 27
2 of 4
16
count: 27
count: 30

,
count: 30
3 of 4
,
count: 30
count: 32

8210
count: 32
count: 37

Aarhus
count: 37
4 of 4
Aarhus
count: 37
count: 44

V
count: 44
count: 46

['O', 'O', 'B-FACILITY', 'B-CARDINAL', 'I-CARDINAL', 'I-LOCATION']


In [27]:
test_idx = 128
text = train.iloc[test_idx]['text']
ents = train.iloc[test_idx]['ents']
print(text)
print(ents)
print(text[13:26])
print(text[27:29])
print(text[31:35])
print(text[36:44])

tokenized_text = bert_tokenizer(text).tokens()
no_special_tokens = tokenized_text[1:-1]
print(tokenized_text)
print(no_special_tokens)
pure_tokens = []

for i, token in enumerate(no_special_tokens):
    first_letter = token[0]

    if first_letter == '#':
        pure_token = token.strip('#')
        combined_token = pure_tokens[-1] + pure_token
        pure_tokens.pop()
        pure_tokens.append(combined_token)

    else:
        pure_tokens.append(token)

print(pure_tokens)

count = 0
skipped_ents = 0
new_ents = []
new_ents_dict = []

for token in pure_tokens:
    # print(f"{i+1} of {len(ents)}")
    tok_len = len(token)
    new_count = count + tok_len

    for ent in ents[skipped_ents:]:
        ent_start = ent['start']
        ent_end = ent['end']
        ent_label = ent['label']
    
        if new_count < ent_start:
            new_ents.append('O')

        elif new_count == ent_start:
            new_ents.append('B-' + ent_label)

        elif count > ent_start and new_count <= ent_end:
            new_ents.append('I-' + ent_label)

        elif new_count == ent_end:
            count = new_count + 1
            skipped_ents += 1
            break

        count = new_count + 1

print(new_ents)

Fysisk butik Bredskiftevej 16, 8210 Aarhus V
[{'start': 13, 'end': 26, 'label': 'FACILITY'}
 {'start': 27, 'end': 29, 'label': 'CARDINAL'}
 {'start': 31, 'end': 35, 'label': 'CARDINAL'}
 {'start': 36, 'end': 44, 'label': 'LOCATION'}]
Bredskiftevej
16
8210
Aarhus V
['[CLS]', 'F', '##ysis', '##k', 'but', '##ik', 'Br', '##eds', '##ki', '##fte', '##vej', '16', ',', '821', '##0', 'Aarhus', 'V', '[SEP]']
['F', '##ysis', '##k', 'but', '##ik', 'Br', '##eds', '##ki', '##fte', '##vej', '16', ',', '821', '##0', 'Aarhus', 'V']
['Fysisk', 'butik', 'Bredskiftevej', '16', ',', '8210', 'Aarhus', 'V']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-CARDINAL', 'O', 'B-LOCATION', 'I-LOCATION']


In [7]:
for row in train.iloc:
    text = row["text"]
    ents = row["ents"]

    token_start = 0

    tokenized_text = bert_tokenizer(text).tokens()
    no_special_tokens = tokenized_text[1:-1]
    pure_tokens = []

    for i, token in enumerate(no_special_tokens):
        first_letter = token[0]

        if first_letter == '#':
            pure_token = token.strip('#')
            combined_token = pure_tokens[-1] + pure_token
            pure_tokens.pop()
            pure_tokens.append(combined_token)

        else:
            pure_tokens.append(token)

    
    new_ents = []
    for ent in ents:
        if token in ("[CLS]", "[SEP]"):
            # skip to next token
            continue 
        if token.startswith("##"):
            token = token[2:]

IndexError: list index out of range

In [ ]:
tokenized_text = bert_tokenizer(train['text'][3]).tokens()
no_special_tokens = tokenized_text[1:-1]

pure_tokens = []
for i, token in enumerate(no_special_tokens):
    first_letter = token[0]
    if first_letter == '#':
        pure_token = token.strip('#')
        combined_token = pure_tokens[-1] + pure_token
        pure_tokens.pop()
        pure_tokens.append(combined_token)
    else:
        pure_tokens.append(token)

['[CLS]', 'Vi', 'har', 'også', '360', 'grader', '##s', 'Private', 'Bank', '##ing', 'råd', '##gi', '##vning', 'til', 'form', '##uen', '##de', 'kunde', '##r', 'med', 'specialist', '##er', 'inden', '##for', 'ska', '##t', 'og', 'in', '##vest', '##ering', '[SEP]']
['Vi', 'har', 'også', '360', 'grader', '##s', 'Private', 'Bank', '##ing', 'råd', '##gi', '##vning', 'til', 'form', '##uen', '##de', 'kunde', '##r', 'med', 'specialist', '##er', 'inden', '##for', 'ska', '##t', 'og', 'in', '##vest', '##ering']
['Vi', 'har', 'også', '360', 'graders', 'Private', 'Banking', 'rådgivning', 'til', 'formuende', 'kunder', 'med', 'specialister', 'indenfor', 'skat', 'og', 'investering']


In [ ]:
ents = train['ents'][3]
print(ents)
ent = ents[0]
print(ent)
new_ents = []

count = 0
start = ent['start']
end = ent['end']
tag = ent['label']

for token in pure_tokens:
    new_ent_dict = {}
    token_len = len(token)
    new_count = count + token_len

    if count == start and new_count <= end:
        new_ent_dict['start'] = count
        new_ent_dict['end'] = count + new_count
        new_ent_dict['label'] = 'B-' + tag

    elif count > start and new_count < end:
        new_ent_dict['start'] = count
        new_ent_dict['end'] = count + new_count
        new_ent_dict['label'] = 'I-' + tag
    
    count += tok

SyntaxError: invalid syntax (2055908776.py, line 8)

In [ ]:
for i in range(10):
    for j in range(3):
        if

['ko', 'hest']